In [328]:
# Importo los paquetes
import pandas as pd
import numpy as np
import sqlite3
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestClassifier # el modelo
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix # para ver cómo funciona
from sklearn.preprocessing import LabelEncoder # poner los datos tipo int



In [329]:
# Importo los dataframes
with sqlite3.connect("../laliga.sqlite") as conn:
        df = pd.read_sql("SELECT * FROM Matches", conn)
conn.close() 

In [330]:
# Importo los dataframes
with sqlite3.connect("clasification.sqlite") as conn:
        df_teams = pd.read_sql("SELECT * FROM clasification", conn)
conn.close()  

In [331]:
# Si pongo directamente df_games =con(...) me da error en la siguiente celda
df_games=df  

In [332]:
# Le creo a df_games la columna de result (home, visitor, tie) que va a ser mi variable Y en el modelo
df_games["result"] = None

df_games.loc[(df_games["score"].str.split(":").str[0]) > (df_games["score"].str.split(":").str[1]), "result"] = 'home'
df_games.loc[(df_games["score"].str.split(":").str[0]) == (df_games["score"].str.split(":").str[1]), "result"] = 'tie'
df_games.loc[(df_games["score"].str.split(":").str[0]) < (df_games["score"].str.split(":").str[1]), "result"] = 'visitor'

In [333]:
df_games.drop(["date", "time", "score"], axis=1, inplace=True)
df_games

,season,division,matchday,home_team,away_team,result
0,1928-1929,1,1,Arenas Club,Athletic Madrid,visitor
1,1928-1929,1,1,Espanyol,Real Unión,home
2,1928-1929,1,1,Real Madrid,Catalunya,home
3,1928-1929,1,1,Donostia,Athletic,tie
4,1928-1929,1,1,Racing,Barcelona,visitor
...,...,...,...,...,...,...
48775,2021-2022,2,42,Real Oviedo,UD Ibiza,None
48776,2021-2022,2,42,Real Sociedad B,Real Zaragoza,None
48777,2021-2022,2,42,Sporting Gijón,UD Las Palmas,None
48778,2021-2022,2,42,CD Tenerife,FC Cartagena,None


In [334]:
df_games_home = df_games[["season", "division", "matchday", "home_team", "result"]]
df_games_visitor = df_games[["season", "division", "matchday", "away_team", "result"]]
df_games_home

,season,division,matchday,home_team,result
0,1928-1929,1,1,Arenas Club,visitor
1,1928-1929,1,1,Espanyol,home
2,1928-1929,1,1,Real Madrid,home
3,1928-1929,1,1,Donostia,tie
4,1928-1929,1,1,Racing,visitor
...,...,...,...,...,...
48775,2021-2022,2,42,Real Oviedo,None
48776,2021-2022,2,42,Real Sociedad B,None
48777,2021-2022,2,42,Sporting Gijón,None
48778,2021-2022,2,42,CD Tenerife,None


In [335]:
division = 1
season = "2019-2020"
matchday = 3
season_train = "1990:2010"


In [336]:
# Luego, aplicar las operaciones de cadena de caracteres
df_teams_train = df_teams

# Asegurarte de que la columna "season" del DataFrame sea de tipo cadena de caracteres
df_teams_train["season"] = df_teams_train["season"].astype(str)

df_teams_train = df_teams_train[(df_teams_train["season"].str.split("-").str[0] >= season_train.split(":")[0]) &
                    (df_teams_train["season"].str.split("-").str[1] <= season_train.split(":")[1])]
df_teams_train


,season,division,rank,team,matchday,GF,GA,GD,W,L,T,Pts
20208,2009-2010,1,1.0,Getafe,1,4.0,1.0,3.0,1.0,0.0,0.0,3.0
20209,2009-2010,1,2.0,Barcelona,1,3.0,0.0,3.0,1.0,0.0,0.0,3.0
20210,2009-2010,1,3.0,Málaga CF,1,3.0,0.0,3.0,1.0,0.0,0.0,3.0
20211,2009-2010,1,4.0,RCD Mallorca,1,2.0,0.0,2.0,1.0,0.0,0.0,3.0
20212,2009-2010,1,5.0,Valencia,1,2.0,0.0,2.0,1.0,0.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...
53057,1990-1991,2,16.0,Celta de Vigo,38,31.0,38.0,-7.0,8.0,10.0,20.0,44.0
53058,1990-1991,2,17.0,Palamós CF,38,33.0,46.0,-13.0,9.0,12.0,17.0,44.0
53059,1990-1991,2,18.0,UD Salamanca,38,41.0,40.0,1.0,9.0,16.0,13.0,40.0
53060,1990-1991,2,19.0,Levante,38,27.0,51.0,-24.0,6.0,17.0,15.0,33.0


In [337]:
df_games_home_train = df_games_home
# Asegurarte de que la columna "season" del DataFrame sea de tipo cadena de caracteres

df_games_home_train["season"] = df_games_home_train["season"].astype(str)

# Luego, aplicar las operaciones de cadena de caracteres
df_games_home_train = df_games_home_train[(df_games_home_train["season"].str.split("-").str[0] >= season_train.split(":")[0]) &
                    (df_games_home_train["season"].str.split("-").str[1] <= season_train.split(":")[1])]
df_games_home_train


/tmp/ipykernel_33237/4240325088.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_games_home_train["season"] = df_games_home_train["season"].astype(str)


,season,division,matchday,home_team,result
13874,1990-1991,1,1,Espanyol,visitor
13875,1990-1991,1,1,CD Logroñés,visitor
13876,1990-1991,1,1,Real Madrid,home
13877,1990-1991,1,1,Real Valladolid,tie
13878,1990-1991,1,1,Real Betis,tie
...,...,...,...,...,...
42851,2009-2010,2,42,Cádiz CF,home
42852,2009-2010,2,42,Celta de Vigo,visitor
42853,2009-2010,2,42,Elche CF,home
42854,2009-2010,2,42,UD Las Palmas,home


In [338]:
df_games_visitor_train = df_games_visitor
# Asegurarte de que la columna "season" del DataFrame sea de tipo cadena de caracteres

df_games_visitor_train["season"] = df_games_visitor_train["season"].astype(str)

# Luego, aplicar las operaciones de cadena de caracteres
df_games_visitor_train = df_games_visitor_train[(df_games_visitor_train["season"].str.split("-").str[0] >= season_train.split(":")[0]) &
                    (df_games_visitor_train["season"].str.split("-").str[1] <= season_train.split(":")[1])]
df_games_visitor_train

/tmp/ipykernel_33237/3908699393.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_games_visitor_train["season"] = df_games_visitor_train["season"].astype(str)


,season,division,matchday,away_team,result
13874,1990-1991,1,1,Barcelona,visitor
13875,1990-1991,1,1,RCD Mallorca,visitor
13876,1990-1991,1,1,CD Castellón,home
13877,1990-1991,1,1,CA Osasuna,tie
13878,1990-1991,1,1,Sporting Gijón,tie
...,...,...,...,...,...
42851,2009-2010,2,42,CD Numancia,home
42852,2009-2010,2,42,SD Huesca,visitor
42853,2009-2010,2,42,Real Sociedad,home
42854,2009-2010,2,42,Gimnàstic,home


In [339]:
df_train_home = pd.merge(df_games_home_train, df_teams_train, how="outer", 
                    left_on=['home_team', "season", "division", "matchday"],
                    right_on=['team', "season", "division", "matchday"], 
                    suffixes=("_home", ""))

df_train_home.dropna(subset="home_team", inplace=True)

In [340]:
df_train_away = pd.merge(df_games_visitor_train, df_teams_train, 
                    left_on=['away_team', "season", "division", "matchday"],
                    right_on=['team', "season", "division", "matchday"], 
                    how='outer', suffixes=("_away",""))
df_train_away.dropna(subset="away_team", inplace=True)

In [341]:
df_train = pd.concat([df_train_home, df_train_away])
df_train

,season,division,matchday,home_team,result,rank,team,GF,GA,GD,W,L,T,Pts,away_team
0,1990-1991,1,1,Espanyol,visitor,19.0,Espanyol,0.0,1.0,-1.0,0.0,1.0,0.0,0.0,NaN
1,1990-1991,1,1,CD Logroñés,visitor,15.0,CD Logroñés,1.0,2.0,-1.0,0.0,1.0,0.0,0.0,NaN
2,1990-1991,1,1,Real Madrid,home,5.0,Real Madrid,1.0,0.0,1.0,1.0,0.0,0.0,3.0,NaN
3,1990-1991,1,1,Real Valladolid,tie,11.0,Real Valladolid,1.0,1.0,0.0,0.0,0.0,1.0,1.0,NaN
4,1990-1991,1,1,Real Betis,tie,7.0,Real Betis,2.0,2.0,0.0,0.0,0.0,1.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16425,2009-2010,2,42,NaN,home,8.0,CD Numancia,55.0,53.0,2.0,16.0,15.0,11.0,59.0,CD Numancia
16426,2009-2010,2,42,NaN,visitor,13.0,SD Huesca,36.0,40.0,-4.0,12.0,14.0,16.0,52.0,SD Huesca
16427,2009-2010,2,42,NaN,home,1.0,Real Sociedad,53.0,37.0,16.0,20.0,8.0,14.0,74.0,Real Sociedad
16428,2009-2010,2,42,NaN,home,18.0,Gimnàstic,42.0,55.0,-13.0,14.0,19.0,9.0,51.0,Gimnàstic


In [342]:
# Supongamos que tienes un DataFrame df con una columna 'etiqueta' de tipo 'object'
le = LabelEncoder()
df_train['prediction'] = le.fit_transform(df_train['result'])
#df_train["home"] = le.fit_transform(df_train["home_team"])

In [343]:
# Selecciona las características (X_train) y etiquetas (y_train)
X_train = df_train[['rank', 'GD', "W", "Pts"]]  
y_train = df_train['prediction']  

In [344]:
df_teams_test = df_teams[(df_teams["season"] == season) & (df_teams["division"] == division) & (df_teams["matchday"] == matchday)]

In [345]:
df_games_home_test = df_games_home[(df_games_home["season"] == season) & 
                                   (df_games_home["division"] == division) & 
                                   (df_games_home["matchday"] == matchday)]

df_games_home_test


,season,division,matchday,home_team,result
25078,2019-2020,1,3,Sevilla FC,tie
25079,2019-2020,1,3,Athletic,home
25080,2019-2020,1,3,CA Osasuna,tie
25081,2019-2020,1,3,Getafe,tie
25082,2019-2020,1,3,Levante,home
25083,2019-2020,1,3,Real Betis,home
25084,2019-2020,1,3,Valencia,home
25085,2019-2020,1,3,Atlético Madrid,home
25086,2019-2020,1,3,Espanyol,visitor
25087,2019-2020,1,3,Villarreal,tie


In [346]:
df_games_visitor_test = df_games_visitor[(df_games_visitor["season"] == season) & 
                                   (df_games_visitor["division"] == division) & 
                                   (df_games_visitor["matchday"] == matchday)]

df_games_visitor_test


,season,division,matchday,away_team,result
25078,2019-2020,1,3,Celta de Vigo,tie
25079,2019-2020,1,3,Real Sociedad,home
25080,2019-2020,1,3,Barcelona,tie
25081,2019-2020,1,3,Alavés,tie
25082,2019-2020,1,3,Real Valladolid,home
25083,2019-2020,1,3,CD Leganés,home
25084,2019-2020,1,3,RCD Mallorca,home
25085,2019-2020,1,3,SD Eibar,home
25086,2019-2020,1,3,Granada CF,visitor
25087,2019-2020,1,3,Real Madrid,tie


In [347]:
df_test_home = pd.merge(df_games_home_test, df_teams_test, how="outer", 
                    left_on=['home_team', "season", "division", "matchday"],
                    right_on=['team', "season", "division", "matchday"], 
                    suffixes=("_home", ""))

df_test_home.dropna(subset="home_team", inplace=True)
df_test_home

,season,division,matchday,home_team,result,rank,team,GF,GA,GD,W,L,T,Pts
0,2019-2020,1,3,Sevilla FC,tie,3.0,Sevilla FC,4.0,1.0,3.0,2.0,0.0,1.0,7.0
1,2019-2020,1,3,Athletic,home,2.0,Athletic,4.0,1.0,3.0,2.0,0.0,1.0,7.0
2,2019-2020,1,3,CA Osasuna,tie,6.0,CA Osasuna,3.0,2.0,1.0,1.0,0.0,2.0,5.0
3,2019-2020,1,3,Getafe,tie,17.0,Getafe,2.0,3.0,-1.0,0.0,1.0,2.0,2.0
4,2019-2020,1,3,Levante,home,4.0,Levante,4.0,2.0,2.0,2.0,1.0,0.0,6.0
5,2019-2020,1,3,Real Betis,home,15.0,Real Betis,5.0,8.0,-3.0,1.0,2.0,0.0,3.0
6,2019-2020,1,3,Valencia,home,10.0,Valencia,3.0,2.0,1.0,1.0,1.0,1.0,4.0
7,2019-2020,1,3,Atlético Madrid,home,1.0,Atlético Madrid,5.0,2.0,3.0,3.0,0.0,0.0,9.0
8,2019-2020,1,3,Espanyol,visitor,19.0,Espanyol,0.0,5.0,-5.0,0.0,2.0,1.0,1.0
9,2019-2020,1,3,Villarreal,tie,16.0,Villarreal,7.0,8.0,-1.0,0.0,1.0,2.0,2.0


In [348]:
df_test_away = pd.merge(df_games_visitor_test, df_teams_test, 
                    left_on=['away_team', "season", "division", "matchday"],
                    right_on=['team', "season", "division", "matchday"], 
                    how='outer', suffixes=("_away",""))
df_test_away.dropna(subset="away_team", inplace=True)

In [349]:
df_test = pd.concat([df_test_home, df_test_away])
df_test

,season,division,matchday,home_team,result,rank,team,GF,GA,GD,W,L,T,Pts,away_team
0,2019-2020,1,3,Sevilla FC,tie,3.0,Sevilla FC,4.0,1.0,3.0,2.0,0.0,1.0,7.0,NaN
1,2019-2020,1,3,Athletic,home,2.0,Athletic,4.0,1.0,3.0,2.0,0.0,1.0,7.0,NaN
2,2019-2020,1,3,CA Osasuna,tie,6.0,CA Osasuna,3.0,2.0,1.0,1.0,0.0,2.0,5.0,NaN
3,2019-2020,1,3,Getafe,tie,17.0,Getafe,2.0,3.0,-1.0,0.0,1.0,2.0,2.0,NaN
4,2019-2020,1,3,Levante,home,4.0,Levante,4.0,2.0,2.0,2.0,1.0,0.0,6.0,NaN
5,2019-2020,1,3,Real Betis,home,15.0,Real Betis,5.0,8.0,-3.0,1.0,2.0,0.0,3.0,NaN
6,2019-2020,1,3,Valencia,home,10.0,Valencia,3.0,2.0,1.0,1.0,1.0,1.0,4.0,NaN
7,2019-2020,1,3,Atlético Madrid,home,1.0,Atlético Madrid,5.0,2.0,3.0,3.0,0.0,0.0,9.0,NaN
8,2019-2020,1,3,Espanyol,visitor,19.0,Espanyol,0.0,5.0,-5.0,0.0,2.0,1.0,1.0,NaN
9,2019-2020,1,3,Villarreal,tie,16.0,Villarreal,7.0,8.0,-1.0,0.0,1.0,2.0,2.0,NaN


In [350]:
le = LabelEncoder()
df_test['prediction'] = le.fit_transform(df_test['result'])
#df_test["home"] = le.fit_transform(df_test["home_team"])
df_test

,season,division,matchday,home_team,result,rank,team,GF,GA,GD,W,L,T,Pts,away_team,prediction
0,2019-2020,1,3,Sevilla FC,tie,3.0,Sevilla FC,4.0,1.0,3.0,2.0,0.0,1.0,7.0,NaN,1
1,2019-2020,1,3,Athletic,home,2.0,Athletic,4.0,1.0,3.0,2.0,0.0,1.0,7.0,NaN,0
2,2019-2020,1,3,CA Osasuna,tie,6.0,CA Osasuna,3.0,2.0,1.0,1.0,0.0,2.0,5.0,NaN,1
3,2019-2020,1,3,Getafe,tie,17.0,Getafe,2.0,3.0,-1.0,0.0,1.0,2.0,2.0,NaN,1
4,2019-2020,1,3,Levante,home,4.0,Levante,4.0,2.0,2.0,2.0,1.0,0.0,6.0,NaN,0
5,2019-2020,1,3,Real Betis,home,15.0,Real Betis,5.0,8.0,-3.0,1.0,2.0,0.0,3.0,NaN,0
6,2019-2020,1,3,Valencia,home,10.0,Valencia,3.0,2.0,1.0,1.0,1.0,1.0,4.0,NaN,0
7,2019-2020,1,3,Atlético Madrid,home,1.0,Atlético Madrid,5.0,2.0,3.0,3.0,0.0,0.0,9.0,NaN,0
8,2019-2020,1,3,Espanyol,visitor,19.0,Espanyol,0.0,5.0,-5.0,0.0,2.0,1.0,1.0,NaN,2
9,2019-2020,1,3,Villarreal,tie,16.0,Villarreal,7.0,8.0,-1.0,0.0,1.0,2.0,2.0,NaN,1


In [351]:
X_test = df_test[['rank', 'GD', "W", "Pts"]] # he cogido estás aleatoriamente
y_test = df_test['prediction']

In [352]:
# Crear y entrenar un modelo de clasificación (por ejemplo, Random Forest)
model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier()

In [353]:
# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)
y_pred

array([2, 1, 1, 1, 0, 2, 1, 2, 0, 1, 0, 0, 2, 1, 1, 0, 0, 0, 0, 1])

In [354]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
confusion_mat = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred, target_names=['home_win(1)', 'visitor_win(2)', 'tie(X)'])

print(f"Accuracy: {accuracy}")
print(f"Confusion Matrix:\n{confusion_mat}")
print(f"Classification Report:\n{class_report}")

Accuracy: 0.5
Confusion Matrix:
[[5 3 2]
 [1 5 2]
 [2 0 0]]
Classification Report:
                precision    recall  f1-score   support

   home_win(1)       0.62      0.50      0.56        10
visitor_win(2)       0.62      0.62      0.62         8
        tie(X)       0.00      0.00      0.00         2

      accuracy                           0.50        20
     macro avg       0.42      0.38      0.39        20
  weighted avg       0.56      0.50      0.53        20

